## Imports

In [5]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:90% !important; }</style>"))

In [125]:
# To embed plots in the notebooks
%matplotlib inline
import matplotlib.pyplot as plt

import numpy as np # numpy library
import scipy . linalg as lng # linear algebra from scipy library
from scipy . spatial import distance # load distance function
from sklearn import preprocessing as preproc # load preprocessing function
from sklearn.linear_model import LinearRegression

# seaborn can be used to "prettify" default matplotlib plots by importing and setting as default
import seaborn as sns
sns.set() # Set searborn as default

## Load dataset

In [3]:
diabetPath = './DiabetesDataNormalized.txt'
T = np.loadtxt(diabetPath, delimiter = ' ', skiprows = 1)
y = T[:, 10]
X = T[:,:10]

# Get number of observations (n) and number of independent variables (p)
[n, p] = np.shape(X)

M = X

## 1 Solve the Ordinary Least Squares (OLS) computationally (for the diabetes data set):

> (a) What is the difference between using a brute force implementation fo an OLS solver and a numerically ’smarter’ implementation? Compute the ordinary least squares solution to the diabetes data set for both options and look at the relative difference. Use for example lng.lstsq to invert the matrix or to solve the linear system of equation.

In [57]:
def calculate_error(X, y, b):
    cross = X.dot(b)
    error = sum((y - cross)**2)
    return error

In [161]:
def ols_solver(X, y):
    stable = np.zeros(X.shape[1])
    beta = np.zeros(X.shape[1])
    learning_rate = 0.001
    error_last = 0
    error_curr = 0
    n_prog = 0
    while (stable == np.full(X.shape[1],0)).any():
        for n,variable in enumerate(beta):
            beta_dict = {'u':np.copy(beta),'l':np.copy(beta),'c':np.copy(beta)}
            error_dict = {'u':0,'l':0,'c':0}
            beta_dict['u'][n] += learning_rate
            beta_dict['l'][n] -= learning_rate
            error_dict['u'] = calculate_error(X, y, beta_dict['u'])
            error_dict['l'] = calculate_error(X, y, beta_dict['l'])
            error_dict['c'] = calculate_error(X, y, beta_dict['c'])
            opt_ind = min({x:error_dict[x] for x in error_dict}, key=error_dict.get)
            beta[n] = beta_dict[opt_ind][n]
            if opt_ind == 'c':
                stable[n] = 1
            else:
                stable[n] = 0
            error_last = error_curr
            error_curr = error_dict[opt_ind]
            
            if error_curr < error_last - 0.000001:
                n_prog = 0
            else:
                n_prog += 1
        if n_prog > 100:
            learning_rate = (learning_rate-1)/2 + 1
        if learning_rate < 0.00000001:
            print('lr')
            break
    
    return beta




def ols_numerical(X, y):
    # Implement the numerical way of calculating the betas 
    return np.linalg.inv((X.T.dot(X))).dot(X.T).dot(y)


In [160]:
# Use the two methods you just created to calculate the betas and inspect the relative difference
beta_bad = ols_solver(X,y)
beta_good = ols_numerical(X,y)



> (b) How could you include an intercept term in Python? This means using the model: $y = β_0 +xβ_1 +...+x_pβ_p +ε $ rather than: $ y=x_1β_1 +...+x_pβ_p +ε. $

In [155]:
tmp = [[1,2,3],[1,2,3]]
np.pad(tmp,((1,0),(0,1)),mode='constant',constant_values=1)

array([[1, 1, 1, 1],
       [1, 2, 3, 1],
       [1, 2, 3, 1]])

In [157]:
# Include offset / intercept
X_new = np.pad(X,((0,0),(0,1)),mode='constant',constant_values=1)

beta_bad = ols_solver(X_new,y)
beta_good = ols_numerical(X_new,y)
print(beta_good)
print(beta_bad)

[-6.18292545e-03 -1.48130075e-01  3.21100050e-01  2.00366920e-01
 -4.89313521e-01  2.94473646e-01  6.24127211e-02  1.09368973e-01
  4.64049083e-01  4.17718663e-02 -6.40112963e-16]
[-0.006 -0.148  0.322  0.2   -0.439  0.255  0.039  0.102  0.445  0.042
  0.   ]


> (c) Calculate the mean squared error $MSE = 1/n \sum^n_{i=1} (_i−x_iβ)^2$.

In [163]:
# Calculate the estimated y values for the solver and numerical OLS and use these to calculate the MSE.
np.mean((y-X.dot(beta_good))**2)

0.4811605108615969

> (d) Calculate the residual sum of squares $RSS = ∥{\bf y} − Xβ∥_2^2$ and the total sum of squares $T SS = ∥{\bf y} − y ̄∥_2^2$, where $y ̄$ is the estimated mean of ${\bf y}$. Report on the $R^2$ measure, that is, the proportion of variance in the sample set explained by the
  model: $R^2 = 1 − \frac{RSS}{TSS}$

In [167]:
RSS = np.sum((y-X.dot(beta_good))**2)
TSS = np.sum((y-np.mean(X.dot(beta_good)))**2)
R2 = 1-RSS/TSS

In [168]:
R2

0.5177484222203499